<div style="
  padding: 30px;
  text-align: center;" class='row'>
<div style="float:left;width: 15%;" class='column'><a href="https://www.colombiacompra.gov.co"><img alt="Logo Colombia Compra Eficiente" id="logocce" src="https://www.colombiacompra.gov.co/sites/cce_public/files/Images/logocce.png" style="height: 45px;"></a></div>
    <div style="float:left;width: 70%;" class='column'>
        <h1> Conexión al DataLake de Colombia Compra Eficiente EMAE
        </h1> 
    </div>
<div style="float:left;width: 15%;" class='column'><a href="https://www.dnp.gov.co/" target="_blank"><img class="float-right" id="logodnp" src="https://www.dnp.gov.co/img/logoNuevo.jpg" style="width: 200px;"></a></div>
</div>

## 1. IDENTIFICACIÓN DEL INSUMO

|||
|:--|:--|
|**Fecha**|Mayo 2023|
|**Ciudad**|Bogotá D.C.|
|**Esquema de presentación del insumo**|Cuaderno Jupyter|
|**Título del insumo**| **Conexión al DataLake**|
|**Descripción y alcance**|Script para la consulta de recursos en el datalake para EMAE.|
|**Periodicidad del insumo**|único|
|**Solicitante**|No aplica|
|**Versión del insumo**|Final|

## 2. DESTINO Y AUTORES DEL INFORME / INSUMO

|||
|:--|:--|
|**Destinatario**|<table align='left'><tr><td>*Nombre:*</td> <td>Equipo analítica EMAE</td></tr> <tr><td>*Cargo:*</td> <td>NA</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE</td></tr></table>|
|**Autores**|<table><tr><td>*Nombre:*</td> <td>Equipo de Datos -GAEC</td></tr><tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|
|**Aprobación**|<table><tr><td>*Nombre:*</td> <td>Ricardo Suarez</td></tr> <tr><td>*Cargo:*</td> <td>Subdirector Estudios de Mercado y Abastecimiento Estratégico</td></tr>  <tr><td>*Área:*</td> <td>Subdirección de estudios de Mercado y Abastecimiento Estratégico – EMAE.</td></tr></table>|

En este cuaderno encontrará un ejemplo de cómo consultar datos del datalake de Colombia Compra Eficiente. Para ello, se utilizará el lenguaje de programación Python y la librería pandas.

Adicionalmente debe tener:

- Python 3.x instalado
- Biblioteca `azure-storage-blob` instalada
- Credenciales de acceso al datalake

## Conexión al datalake

Consulta de archivos de configuración y conexión al datalake.

```python
import os
import pandas as pd
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__
import configparser
from configparser import ExtendedInterpolation
## Cargo las credenciales de acceso al datalake
config = configparser.ConfigParser(interpolation=ExtendedInterpolation())
config.read('../Archivos de configuración/config.ini')

# Obtener los valores de configuración
account_name = config.get('ConexionDatalake', 'account_name')
sas_token = config.get('ConexionDatalake', f'sas_token')
url_blob = config.get('ConexionDatalake', 'url_blob')


# Utilizar los valores de configuración
print(f'Account Name: {account_name}')
print(f'SAS Token: {sas_token}')
print(f'URL Blob: {url_blob}')
```


In [ ]:
import os
import pandas as pd
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__
import configparser
from configparser import ExtendedInterpolation
## Cargo las credenciales de acceso al datalake
config = configparser.ConfigParser(interpolation=ExtendedInterpolation())
config.read('../Archivos de configuración/config.ini')

# Obtener los valores de configuración
account_name = config.get('ConexionDatalake', 'account_name')
sas_token = config.get('ConexionDatalake', f'sas_token')
url_blob = config.get('ConexionDatalake', 'url_blob')


# Utilizar los valores de configuración
print(f'Account Name: {account_name}')
print(f'SAS Token: {sas_token}')
print(f'URL Blob: {url_blob}')

En este caso, usted debe tener un archivo de configuración llamado `config.ini` en la carpeta `Archivos de configuración` con la siguiente información:

```ini
[ConexionDatalake]
account_name = <account_name>
sas_token = <sas_token>
datalake_path = <datalake_path>
url_blob = <url_blob>
```

## Consulta de archivos

Ahora vamos a consultar el archivo limpio que está en el datalake. Para ello, vamos a utilizar la función `list_blobs` de la librería `azure.storage.blob`.

```python
# Crear el cliente del datalake
connect_str = f'DefaultEndpointsProtocol=https;AccountName={account_name};AccountKey={sas_token};EndpointSuffix=core.windows.net'
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

# Obtener el contenedor
container_client = blob_service_client.get_container_client('name_blob')

# Listar los archivos del contenedor

blobs_list = container_client.list_blobs()

# Crear un dataframe con los archivos del contenedor
df = pd.DataFrame(blobs_list)

# Filtrar los archivos que contengan la palabra "limpio"
df = df[df['name'].str.contains('limpio')]

# Mostrar los archivos

display(df)
```


In [ ]:
# Conexión a la cuenta de almacenamiento de Azure utilizando el token SAS
connect_str = f"BlobEndpoint={url_blob};SharedAccessSignature={sas_token}"
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

# Obtener el contenedor
container_client = blob_service_client.get_container_client('datosemae')

# Listar los archivos del contenedor

blobs_list = container_client.list_blobs()

blobs_list=list(blobs_list)

## Crear un dataframe con los archivos del contenedor

keys=list(blobs_list[0].keys())

lista=[]
for i in blobs_list:
    lista_simple=[]
    for j in keys:
        try:
            lista_simple.append(i[j])
        except:
            lista_simple.append(None)
    lista.append(lista_simple)

df=pd.DataFrame(lista,columns=keys)

df=df[['name','container','size']]

In [ ]:
df

Ahora usemos un archivo de ejemplo para consultar los datos. En este caso, vamos a consultar el archivo `contratos_limpio.csv`.

```python
# Obtener el archivo
blob_client = blob_service_client.get_blob_client(container='name_blob', blob='ruta_file')

# Usar pandas para leer el archivo
df = pd.read_csv(blob_client.download_blob())


# Mostrar los datos
display(df.head())
```




In [ ]:
# Obtener el archivo
blob_client = blob_service_client.get_blob_client(container='name_blob', blob='ruta_file')

# Usar pandas para leer el archivo
df = pd.read_csv(blob_client.download_blob())


# Mostrar los datos
display(df.head())

## Cierre

En este cuaderno se mostró cómo consultar datos del datalake de Colombia Compra Eficiente. Para ello, se utilizó el lenguaje de programación Python y la librería pandas.